# Access Keyvault secrets from a Managed Online Endpoint

In this example we create a Keyvault, set a secret, and then retrieve the secret from a Managed Online Endpoint using the endpoint's system-assigned managed identity. By using the managed identity, the need to pass secrets as well as any other credentials in the image or deployment is avoided.

## Prerequisites: 
* The following additional Python packages should be installed: 
    * [azure-mgmt-keyvault](https://pypi.org/project/azure-mgmt-keyvault/) - Used to create a keyvault
    * [azure-keyvault](https://pypi.org/project/azure-keyvault/)- Used to set the secret and permissions

Install the prerequisites with the following code: 

In [ ]:
%pip install azure-mgmt-keyvault
%pip install azure-keyvault

## 1. Configure parameters, assets, and clients

### 1.1 Set workspace details

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

In [12]:
subscription_id = "6fe1c377-b645-4e8e-b588-52e57cc856b2"
resource_group = "alwallace-rg"
workspace_name = "alwallace"

### 1.2 Set endpoint and keyvault details

In [18]:
import random

rand = random.randint(0, 10000)

endpoint_name = f"endpt-moe-kv-{rand}"
keyvault_name = f"kvexample{rand}"

### 1.3 Set asset paths

In [ ]:
import os

base_path = "keyvault"
code_path = os.path.join(base_path, "code")
conda_file_path = os.path.join(base_path, "env.yml")
sample_request_path = os.path.join(base_path, "sample_request.json")

### 1.4 Examine the scoring script
The scoring script uses a `ManagedIdentityCredential` to authenticate itself to the Keyvault via a `SecretClient` from the `azure-keyvault` package. No arguments are needed to instantiate the credential object when this code is executed in a deployment, because it reads the environment variables `MSI_SECRET` and `MSI_ENDPOINT` which are already present by default.

As part of the deployment, we will pass an environment variable called `KV_SECRET_MULTIPLIER` and give it the `multiplier@https://<VAULT_NAME>.vault.azure.net`. The convenience function `load_secrets` looks for environment variables with `KV_SECRET` and replaces their values with the actual value of the secret from the keyvault.

When a request is received, `input` is multiplied by our secret. 

```python 
from azure.identity import ManagedIdentityCredential
from azure.keyvault.secrets import SecretClient
import os
import json

multiplier : int = None

def load_secrets():
    """
    Replaces the values of environment variables with names containing "KV_SECRET" and values of the form "<SECRET_NAME>@<VAULT_URL>" with the actual secret values

    Uses the ManagedIdentityCredential to create a SecretClient for each <VAULT_URL>. The endpoint's Managed Identity should have the get permission for secrets. 

    Example: 
        KV_SECRET_FOO: foo@https://keyvault123.vault.azure.net

        Will be replaced with the actual vaule of the secret named foo in keyvault123. 
    """
    secret_clients = {}
    credential = ManagedIdentityCredential()

    for k, v in os.environ.items():
        if "KV_SECRET" in k:
            try: 
                secret_name, vault_url = v.split("@")
            except ValueError: 
                raise ValueError(f"Wrong value format for env var {k} with value {v}. Should be of the form <SECRET_NAME>@<VAULT_URL>")

            if vault_url in secret_clients:
                secret_client = secret_clients[vault_url]
            else: 
                secret_client = SecretClient(vault_url=vault_url, credential=credential)
                secret_clients[vault_url] = secret_client

            secret_value = secret_client.get_secret(secret_name).value
            os.environ[k] = secret_value

def init():
    load_secrets() 
    
    global multiplier
    multiplier = int(os.getenv("KV_SECRET_MULTIPLIER"))

def run(data): 
    data = json.loads(data)
    input = data["input"]
    output = input*multiplier

    return {"output" : output}

### 1.5 Create an MLClient instance

In [15]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    CodeConfiguration,
    Environment,
)
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

Class SystemCreatedStorageAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SystemCreatedAcrAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class RegistryOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### 1.6 Create a Keyvault Management client

In [16]:
from azure.mgmt.keyvault import KeyVaultManagementClient

keyvault_mgmt_client = KeyVaultManagementClient(
    credential=credential, subscription_id=subscription_id
)

## 2. Create an endpoint

In [ ]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name)

In [22]:
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

## 3. Create a Keyvault

### 3.1 Get the Endpoint Object ID, location, and Tenant ID

In [28]:
endpoint_object_id = endpoint.identity.principal_id
location = endpoint.location
tenant_id = endpoint.identity.tenant_id

### 3.2 Get your User Object ID

In [ ]:
raise Exception(os.environ.keys())

In [ ]:
from azure.identity import AzureCliCredential, EnvironmentCredential,DefaultAzureCredential
import requests

cli_credential = AzureCliCredential()
token = cli_credential.get_token("https://graph.microsoft.com").token
res = requests.get(
    "https://graph.microsoft.com/v1.0/me", headers={"Authorization": f"Bearer {token}"}
)
user_object_id = res.json().get("id")

### 3.3 Define an AccessPolicy for the Endpoint
Allow the endpoint to get secrets in the Keyvault

In [ ]:
from azure.mgmt.keyvault.models import AccessPolicyEntry, Permissions, SecretPermissions

endpoint_access_policy = AccessPolicyEntry(
    tenant_id=endpoint.identity.tenant_id,
    object_id=endpoint.identity.principal_id,
    permissions=Permissions(secrets=[SecretPermissions.GET]),
)

### 3.4 Define an AccessPolicy for the current user
Allow all secret permissions for the current user or Service Principal

In [ ]:
if user_object_id:
    user_or_sp_access_policy = AccessPolicyEntry(
        tenant_id=endpoint.identity.tenant_id,
        object_id=user_object_id,
        permissions=Permissions(secrets=[SecretPermissions.ALL]),
    )
else: 
    user_or_sp_access_policy = AccessPolicyEntry(
        tenant_id=endpoint.identity.tenant_id,
        object_id=user_object_id,
        permissions=Permissions(secrets=[SecretPermissions.ALL]),
    )

### 3.5 Create the Keyvault

In [21]:
ml_client.online_endpoints.get("endpoint-25589")

ResourceNotFoundError: (ResourceNotFound) The Resource 'Microsoft.MachineLearningServices/workspaces/alwallace/onlineEndpoints/endpoint-25589' under resource group 'alwallace-rg' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix
Code: ResourceNotFound
Message: The Resource 'Microsoft.MachineLearningServices/workspaces/alwallace/onlineEndpoints/endpoint-25589' under resource group 'alwallace-rg' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix

In [37]:
from azure.mgmt.keyvault.models import (
    VaultCreateOrUpdateParameters,
    VaultProperties,
    Sku,
)

poller = keyvault_mgmt_client.vaults.begin_create_or_update(
    vault_name=keyvault_name,
    resource_group_name=resource_group,
    parameters=VaultCreateOrUpdateParameters(
        location=location,
        properties=VaultProperties(
            tenant_id=tenant_id,
            sku=Sku(name="Standard"),
            access_policies=[],
        ),
    ),
)
poller.wait()

### 3.6 Confirm the Keyvault creation was successful

In [44]:
keyvault.as_dict()

{'id': '/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/resourceGroups/alwallace-rg/providers/Microsoft.KeyVault/vaults/kvexample2780',
 'name': 'kvexample2780',
 'type': 'Microsoft.KeyVault/vaults',
 'location': 'eastus2',
 'tags': {},
 'system_data': {'created_by': 'v-alwallace@microsoft.com',
  'created_by_type': 'User',
  'created_at': '2022-11-08T03:47:29.735Z',
  'last_modified_by': 'v-alwallace@microsoft.com',
  'last_modified_by_type': 'User',
  'last_modified_at': '2022-11-08T03:47:29.735Z'},
 'properties': {'tenant_id': '72f988bf-86f1-41af-91ab-2d7cd011db47',
  'sku': {'family': 'A', 'name': 'Standard'},
  'access_policies': [],
  'vault_uri': 'https://kvexample2780.vault.azure.net/',
  'enabled_for_deployment': False,
  'enable_soft_delete': True,
  'soft_delete_retention_in_days': 90,
  'enable_rbac_authorization': False,
  'provisioning_state': 'Succeeded',
  'public_network_access': 'Enabled'}}

In [38]:
status = poller.status()
if status != "Succeeded":
    raise Exception(status)
else:
    print("Endpoint creation succeeded")
    keyvault = poller.result()
    print(keyvault)

Endpoint creation succeeded
{'additional_properties': {}, 'id': '/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/resourceGroups/alwallace-rg/providers/Microsoft.KeyVault/vaults/kvexample2780', 'name': 'kvexample2780', 'type': 'Microsoft.KeyVault/vaults', 'location': 'eastus2', 'tags': {}, 'system_data': <azure.mgmt.keyvault.v2022_07_01.models._models_py3.SystemData object at 0x7ffb4994fd90>, 'properties': <azure.mgmt.keyvault.v2022_07_01.models._models_py3.VaultProperties object at 0x7ffb4994fa00>}


In [42]:
keyvault.properties.create_mode

## 4. Set a Keyvault secret

### 4.1 Create a SecretClient

In [ ]:
from azure.keyvault.secrets import SecretClient

secret_client = SecretClient(
    credential=credential, vault_url=f"https://{keyvault_name}.vault.azure.net"
)

### 4.2 Set a secret

In [ ]:
secret = secret_client.set_secret(name="multiplier", value=str(7))

## 5. Create a Deployment

### 5.1 Define and begin the deployment creation
The environment variable `KV_SECRET_MULTIPLIER` is set to `<SECRET_NAME>@<KEYVAULT_URL>`. In the scoring script, this value is parsed and passed to a SecretClient to retrieve the secret from the Keyvault.

In [ ]:
deployment = ManagedOnlineDeployment(
    name="kvdep",
    endpoint_name=endpoint_name,
    model=Model(path=base_path),
    code_configuration=CodeConfiguration(code=code_path, scoring_script="score.py"),
    environment=Environment(
        conda_file=conda_file_path,
        image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference:latest",
    ),
    environment_variables={
        "KV_SECRET_MULTIPLIER": f"multiplier@https://{keyvault_name}.vault.azure.net"
    },
    instance_type="Standard_DS2_v2",
    instance_count=1,
)
poller = ml_client.online_deployments.begin_create_or_update(deployment)
poller.wait()

### 5.2 Confirm the creation was successful

In [ ]:
status = poller.status()
if status != "Succeeded":
    raise ValueError(status)
else:
    print("Endpoint creation succeeded")
    deployment = poller.result()
    print(deployment)

In [ ]:
endpoint.traffic = {"kvdep": 100}
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

## 6. Test the endpoint
The endpoint returns the value of `input` multiplied by the secret.

In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name, request_file=sample_request_path
)

## 7. Delete assets

### 7.1 Delete the endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)

### 7.2 Delete the keyvault

In [ ]:
keyvault_mgmt_client.vaults.delete(
    resource_group_name=resource_group, vault_name=keyvault_name
)